In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import random as rand
import queue
import csv
from collections import OrderedDict
from IPython.display import clear_output
import csv
import re
from heapq import merge
from sklearn import preprocessing
from operator import itemgetter
import bisect
import pickle
import os
from configparser import ConfigParser
import ast
import sys

from sklearn.datasets import make_circles, make_moons
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from IPython.core.debugger import set_trace
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from IPython.display import display
from os import listdir
from os.path import isfile, join

from src.snapprocess import simulatespectra as sim
from src.snapprocess import process
from src.snaputils import reader

In [2]:
# Temporary config func. Original one in the project.
class config:
    """Define constants"""
    AAMass = {'A': 71.037114, 'C': 103.009185, 'D': 115.026943, 'E': 129.042593, 'F': 147.068414, 'G': 57.021464,
              'H': 137.058912, 'I': 113.084064, 'K': 128.094963, 'L': 113.084064, 'M': 131.040485, 'N': 114.042927,
              'P': 97.052764, 'Q': 128.058578, 'R': 156.101111, 'S': 87.032028, 'T': 101.047679, 'V': 99.068414,
              'W': 186.079313, 'Y': 163.0633}

    H2O = 18.015
    NH3 = 17.031
    PROTON = 1.00727647
    DEFAULT_PARAM_PATH = os.path.join(os.getcwd(), 'config.ini')
    PARAM_PATH = None
    l_config = None


    def get_config(section='input', key=None):
        """Read the configuration parameters and return a dictionary."""

        # If file path is given use it otherwise use default.
        file_path = config.PARAM_PATH if config.PARAM_PATH else config.DEFAULT_PARAM_PATH

        # Read config and convert each value to appropriate type.
        # Only for the first time.
        if not config.l_config:
            config.l_config = dict()
            config_ = ConfigParser()
            assert isinstance(file_path, str)
            config_.read(file_path)
            for section_ in config_.sections():
                config.l_config[section_] = dict()
                for key_ in config_[section_]:
                    try:
                        config.l_config[section_][key_] = ast.literal_eval(config_[section_][key_])
                    except (ValueError, SyntaxError):
                        config.l_config[section_][key_] = config_[section_][key_]

        if section and section in config.l_config:
            if key and key in config.l_config[section]:
                return config.l_config[section][key]
            return config.l_config[section]
        return config.l_config

In [3]:
print(config.get_config(section='input', key='charge'))
print(config.get_config(section='input', key='use_mods'))
print(config.get_config(section='ml', key='batch_size'))
print(config.get_config(section='input', key='num_species'))

3
False
2048
7


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.spec_size = config.get_config(section='input', key='spec_size')
        self.searching = False
        
        self.linear1_1 = nn.Linear(self.spec_size, 1024)
        self.linear1_2 = nn.Linear(1024, 512)
        self.linear1_3 = nn.Linear(512, 256)
        
        self.linear2_1 = nn.Linear(self.spec_size, 1024)
        self.linear2_2 = nn.Linear(1024, 512)
        self.linear2_3 = nn.Linear(512, 256)
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.3)
        #self.dropout3 = nn.Dropout(0.3)
        
    def forward(self, data):
        res = []
#         x = data[:, 0]
#         x = self.linear1_1(x.view(-1, self.spec_size))
#         x = F.relu(x)
#         x = self.dropout2(x)
        #x = self.linear1_2(x)
        #x = F.relu(x)
        #x = self.dropout2(x)
        #x = self.dropout2(x)
        #x = self.linear1_3(x)
        #x = F.relu(x)
        #if not self.searching:
        #    x = F.normalize(x)
        #res.append(x)
        for i in range(data.shape[1]):
            x = data[:, i]
            x = self.linear2_1(x.view(-1, self.spec_size))
            x = F.relu(x)
            #x = torch.tanh(x)
            x = self.dropout1(x)
            x = self.linear2_2(x)
            x = F.relu(x)
            #x = torch.tanh(x)
            #x = self.dropout2(x)
            #x = self.dropout1(x)
            #x = self.linear2_3(x)
            #x = F.relu(x)
            #if not self.searching:
            x = F.normalize(x)
            #x = self.linear1_3(x)
            #x = F.relu(x)
            res.append(x)
        
        return res
    
    def name(self):
        return "Net"

In [6]:
rand.seed(37)

config.AAMass
H2O = 18.01528
PROTON = 1.00727647

specsize = 8000
precursor_tolerance = 3.0
num_psm = 1
charge = 2
use_mods = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = config.get_config(section='ml', key='batch_size')

In [38]:
model = torch.load('models/hard_triplet_98.1%_charge_multi-specie.pt').to(device)
model.eval()
model

Net(
  (linear1_1): Linear(in_features=8000, out_features=1024, bias=True)
  (linear1_2): Linear(in_features=1024, out_features=512, bias=True)
  (linear1_3): Linear(in_features=512, out_features=256, bias=True)
  (linear2_1): Linear(in_features=8000, out_features=1024, bias=True)
  (linear2_2): Linear(in_features=1024, out_features=512, bias=True)
  (linear2_3): Linear(in_features=512, out_features=256, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
)

In [7]:
def runModel(loader):
    with torch.no_grad():
        accurate_labels = 0
        all_labels = 0
        loss = 0
        out = torch.Tensor().to(device)
        for batch_idx, data in enumerate(loader):
            data = data.to(device)    
            out = torch.cat((out, model(data)[0]), dim=0)
    print(out.size())
    return out

In [8]:
def readmgfs(folderPath, charge=None):
    
    mgffiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f)) and f.split('.')[-1] == 'mgf']
    assert len(mgffiles) > 0
    
    spectra = []
    masses = []
    charges = []
    for file in mgffiles:
        f = open(join(folderPath, file))
        speclines = f.readlines()
        f.close()
        
        if not speclines:
            continue
            
        spec = np.zeros(specsize)
        isMass = isCharge = isSpec = False
        i = 0
        '''Read Headers'''
        while True:
            line = speclines[i]
            i += 1
            
            splits = line.split('=')
            if splits[0].upper() == 'PEPMASS':
                masses.append(float(splits[1].split(' ')[0]))
                isMass = True
                
            if isMass and splits[0].upper() == 'CHARGE':
                l_charge = int(splits[1][0])
                if charge and l_charge != charge:
                    del masses[-1]
                    isMass = False
                    isCharge = False
                else:
                    charges.append(l_charge)
                    isCharge = True
                break
        
        '''Read Spectrum'''
        while isMass and isCharge and i < len(speclines):
            line = speclines[i]
            i += 1
            
            if line != '\n' and 'END IONS' not in line.upper():
                splits = line.split(' ')
                moz, intensity = float(splits[0]), float(splits[1])
                spec[round(moz)] += round(intensity)
            elif 'END IONS' in line.upper():
                break
                
        if isMass and isCharge:        
            spec = np.clip(spec, None, 1000.0)
            spec = preprocessing.scale(spec)
            spectra.append(spec)
        
    return spectra, masses, charges

In [9]:
def readms2(file, charge=None):
    f = open(file)
    lines = f.readlines()
    f.close()
    
    spectra = []
    masses = []
    charges = []
    i = 0
    while i < len(lines):
        line = lines[i][:-1]
        i += 1
        
        splits = line.split('\t')
        if splits[0] == 'Z' and (charge is None or float(splits[1]) == charge):
            charges.append(float(splits[1]))
            masses.append(float(splits[2]))
            spec = np.zeros(specsize)
            while i < len(lines):
                line = lines[i][:-1] #remove the \n character
                i += 1
                splits = line.split(' ')
                if 'S' in splits[0]:
                    break
                if 'Z' in splits[0]:
                    continue
                moz, intensity = float(splits[0]), float(splits[1])
                spec[round(moz)] += round(intensity)
            
            spec = np.clip(spec, None, 1000.0)
            spec = preprocessing.scale(spec)
            spectra.append(spec)
            
    return spectra, masses, charges

In [10]:
ms2file = '/disk/raptor/lclhome/mtari008/Proteomics/crux/specs/demo.ms2'

#queryspectra, spectramasses, _ = readmgfs(specsPath, charge=2)
queryspectra, spectramasses, _ = readms2(ms2file, None)
with torch.no_grad():
    queryspectraTensor = torch.tensor(queryspectra, dtype=torch.float)[:, None, :]
    queryspectra_loader = torch.utils.data.DataLoader(dataset=queryspectraTensor, batch_size=batch_size, shuffle=False)

    queryspectra_out = runModel(queryspectra_loader)
del queryspectraTensor
del queryspectra_loader
del queryspectra

torch.Size([150, 512])


In [11]:
print(type(queryspectra_out))

<class 'torch.Tensor'>


In [12]:
print(batch_size)

2048


In [13]:
def fastatospectra(lines, start, count, dh):
    tspectra = []
    masses = []
    peps = []
    
    new = prev = 0
    end = min(start+count, len(lines))
    for i, line in enumerate(lines[start:end]):
        splits = line.split('\t')
        
        pep = splits[0]
        #print(pep)
        peps.append(pep)
        spec = sim.get_spectrum(pep)
        tspectra.append(preprocessing.scale(spec))
        masses.append(float(splits[1]))
        
        #print(splits[1])
        '''Progress Monitor'''
        new = int(((i+start)/len(lines)) * 100)
        if (new > prev):
            dh.update(str(new)+'%')
            prev = new
            
    return tspectra, masses, peps

In [14]:
def fastaToSpectraBatch(filePath, spectra_batch_size):    
    f = open(filePath)
    lines = f.readlines()
    f.close()
    masses = []
    spectra_out = torch.Tensor().to(device)
    peps = []
    
    dh = display('0%', display_id=True)
    
    start = 0
    i = 0
    while start < len(lines):
        print('Batch: ' + str(i))
        i += 1
        
        print('Generating spectra...')
        spectra, l_masses, l_peps = fastatospectra(lines, start, spectra_batch_size, dh)
        masses.extend(l_masses)
        peps.extend(l_peps)
        start = start + spectra_batch_size
        
        with torch.no_grad():
            print('Converting to tensor...')
            '''dtype=torch.float'''
            spectra = np.asarray(spectra)
            spectraTensor = torch.as_tensor(spectra, dtype=torch.float)[:, None, :]
            spectra_loader = torch.utils.data.DataLoader(dataset=spectraTensor, batch_size=batch_size, shuffle=False)
            
            print('Running the model...')
            model_out = runModel(spectra_loader)
            spectra_out = torch.cat((spectra_out, model_out), dim=0)
            
    return spectra_out, masses, peps

In [15]:
spectra_batch_size = 500000
#targetPath = '/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.target.txt'
#decoyPath = '/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.decoy.txt'
targetPath = '/disk/raptor/lclhome/mtari008/Proteomics/crux/crux-output/generate-peptides.target.txt'
decoyPath = '/disk/raptor/lclhome/mtari008/Proteomics/crux/crux-output/generate-peptides.decoy.txt'
targetspectra_out, targetmasses, targetpeptides = fastaToSpectraBatch(targetPath, spectra_batch_size)
decoyspectra_out, decoymasses, decoypeptides = fastaToSpectraBatch(decoyPath, spectra_batch_size)


print(len(targetspectra_out), len(targetmasses))
print(len(decoyspectra_out), len(decoymasses))

'99%'

Batch: 0
Generating spectra...
Converting to tensor...
Running the model...
torch.Size([7103, 512])


'99%'

Batch: 0
Generating spectra...
Converting to tensor...
Running the model...
torch.Size([7103, 512])
7103 7103
7103 7103


In [40]:
nearest_target_distances, nearest_target_indexes = process.pairwise_distances(
    queryspectra_out, targetspectra_out).min(1)

nearest_decoy_distances, nearest_decoy_indexes  = process.pairwise_distances(
    queryspectra_out,  decoyspectra_out).min(1)

nearest_distances, target_decoy = torch.cat((nearest_target_distances.view(1, -1), 
                                             nearest_decoy_distances.view(1, -1)), dim=0).min(0)

print(target_decoy.to('cpu'))

sorted_nearest_distances, snd_index = nearest_distances.sort()
target_decoy = target_decoy[snd_index]

print(target_decoy)

## To lookup a peptide corresponding to a value in sorted_nearest_distances at index i:
## td = target_deocy[i]            # a binary array. 0: target, 1: decoy
## query_index = snd_index[i]        # lookup the index of query spectrum
## pep_index = nearest_decoy_indexes[query_index] if td else nearest_target_indexes[query_index]
## td_peptide = decoy_peptides[pep_index] if td else target_peptides[pep_index]

tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 0, 1, 1, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        1, 1,

In [60]:
def fdr(td, target_fdr):
    target_count = 0
    decoy_count = 0
    q_values = [0] * len(td)
    for idx, val in enumerate(td):
        if val == 0:
            target_count += 1
        else:
            decoy_count += 1
        return_fdr = decoy_count / (idx + 1)
        print('{} / {} = {}'.format(decoy_count, idx+1, decoy_count/(idx+1)))
        #if return_fdr > target_fdr:
        #    return target_count, return_fdr
    return target_count, return_fdr

In [61]:
print(fdr(target_decoy, 0.01))

0 / 1 = 0.0
0 / 2 = 0.0
0 / 3 = 0.0
0 / 4 = 0.0
0 / 5 = 0.0
0 / 6 = 0.0
0 / 7 = 0.0
0 / 8 = 0.0
0 / 9 = 0.0
0 / 10 = 0.0
0 / 11 = 0.0
0 / 12 = 0.0
0 / 13 = 0.0
0 / 14 = 0.0
0 / 15 = 0.0
0 / 16 = 0.0
0 / 17 = 0.0
0 / 18 = 0.0
0 / 19 = 0.0
0 / 20 = 0.0
0 / 21 = 0.0
0 / 22 = 0.0
0 / 23 = 0.0
0 / 24 = 0.0
0 / 25 = 0.0
0 / 26 = 0.0
0 / 27 = 0.0
0 / 28 = 0.0
0 / 29 = 0.0
0 / 30 = 0.0
0 / 31 = 0.0
0 / 32 = 0.0
0 / 33 = 0.0
0 / 34 = 0.0
0 / 35 = 0.0
0 / 36 = 0.0
0 / 37 = 0.0
0 / 38 = 0.0
0 / 39 = 0.0
0 / 40 = 0.0
0 / 41 = 0.0
0 / 42 = 0.0
0 / 43 = 0.0
0 / 44 = 0.0
0 / 45 = 0.0
1 / 46 = 0.021739130434782608
2 / 47 = 0.0425531914893617
2 / 48 = 0.041666666666666664
2 / 49 = 0.04081632653061224
3 / 50 = 0.06
3 / 51 = 0.058823529411764705
4 / 52 = 0.07692307692307693
4 / 53 = 0.07547169811320754
4 / 54 = 0.07407407407407407
4 / 55 = 0.07272727272727272
4 / 56 = 0.07142857142857142
4 / 57 = 0.07017543859649122
5 / 58 = 0.08620689655172414
5 / 59 = 0.0847457627118644
5 / 60 = 0.08333333333333333
5 /

In [91]:
print(targetmasses[0:10])
print(len(decoypeptides))
targetspectra_out, targetmasses, targetpeptides = list(zip(*sorted(zip(targetspectra_out, targetmasses, targetpeptides), 
                                                                   key=lambda pair: pair[1])))
decoyspectra_out, decoymasses, decoypeptides = list(zip(*sorted(zip(decoyspectra_out, decoymasses, decoypeptides),
                                                                key=lambda pair: pair[1])))

[787.4672, 4338.1767, 2113.0229, 2570.2763, 2698.3713, 686.4195, 4609.6088, 5249.9746, 968.5523, 1096.6473]
7103


In [ ]:
print(len(targetmasses))
print(targetmasses[1000:1010])
print(decoymasses[1000:1010])

In [ ]:
def find_range(arr, val, tolerance=3):
    assert len(arr) > 0
    assert tolerance > 0
    assert val > 0
    
    left = val - tolerance if val - tolerance >= 0 else 0
    right = val + tolerance
    
    ileft = bisect.bisect_left(arr, left)
    iright = bisect.bisect_right(arr, right)
    
    return (ileft, iright-1) if iright - ileft > 0 else (-1, -1)
    
# Function to insert element 
def insert(lst, n): 
    assert not lst or len(lst[0]) == len(n)
    
    index = num_psm
    # Searching for the position 
    for i in range(len(lst)): 
        if lst[i][0] > n[0]: 
            index = i 
            break
      
    # Inserting n in the list 
    lst = lst[:index] + [n] + lst[index:] 
    return lst[:num_psm]

In [ ]:
def calculate_L2_Dist(queryspectra, querymasses, dbspectra, dbmasses, pre_tol=3, keep_psm=1):
    prev = 0
    psm_scores = []
    psm_pepids = []
    
    for ind, (spec, spec_mass) in enumerate(zip(queryspectra, querymasses)):
        left, right = find_range(dbmasses, spec_mass, pre_tol)
        if left != -1 and right != -1:
            candidate_specs = dbspectra[left:right+1]
            l_scores = np.linalg.norm(np.subtract(candidate_specs, spec), axis=1)**2
            psm_pepids.append(left + l_scores.argsort()[:keep_psm]) #add argsort later
            psm_scores.append(np.sort(l_scores)[:keep_psm])
            #print(ind)
            new = int((ind/len(queryspectra)) * 100)
            if new > prev:
                clear_output(wait=True)
                print(str(new) + '%')
                prev = new
        else:
            print('***********error**************')
            print('ind: ' + str(ind))
            print('spec_mass: ' + str(spec_mass))
            print('**********end error************')
            
    return psm_scores, psm_pepids

In [ ]:
def calculate_xcorr(queryspectra, querymasses, dbspectra, dbmasses, pre_tol=3, keep_psm=1):
    prev = 0
    psm_scores = []
    psm_pepids = []
    
    for ind, (spec, spec_mass) in enumerate(zip(queryspectra, querymasses)):
        left, right = find_range(dbmasses, spec_mass, pre_tol)
        if left != -1 and right != -1:
            candidate_specs = dbspectra[left:right+1]
            l_scores = np.dot(candidate_specs, spec)
            psm_pepids.append(left + l_scores.argsort()[::-1][:keep_psm]) #add argsort later
            psm_scores.append(np.sort(l_scores)[::-1][:keep_psm])
            #print(ind)
            new = int((ind/len(queryspectra)) * 100)
            if new > prev:
                clear_output(wait=True)
                print(str(new) + '%')
                prev = new
        else:
            print('***********error**************')
            print('ind: ' + str(ind))
            print('spec_mass: ' + str(spec_mass))
            print('**********end error************')
            
    return psm_scores, psm_pepids

In [ ]:
# targetpsm_scores, targetpsm_ids = calculate_L2_Dist(queryspectra_out, spectramasses, targetspectra_out,
#                                      targetmasses, precursor_tolerance, num_psm)
# decoypsm_scores, decoypsm_ids = calculate_L2_Dist(queryspectra_out, spectramasses, decoyspectra_out,
#                                      decoymasses, precursor_tolerance, num_psm)
targetpsm_scores, targetpsm_ids = calculate_xcorr(queryspectra_out, spectramasses, targetspectra_out,
                                     targetmasses, precursor_tolerance, num_psm)
decoypsm_scores, decoypsm_ids = calculate_xcorr(queryspectra_out, spectramasses, decoyspectra_out,
                                     decoymasses, precursor_tolerance, num_psm)

In [ ]:
print(targetmasses[2000000:2000010])

In [ ]:
print(len(targetspectra_out))
print(len(decoyspectra_out))

print(len(targetpsm_scores))
print(len(decoypsm_scores))

print(targetpsm_scores[0:10])
print(decoypsm_scores[0:10])

targetpsm_scores = np.asarray(targetpsm_scores).flatten()
decoypsm_scores = np.asarray(decoypsm_scores).flatten()

targetpsm_ids = np.asarray(targetpsm_ids).flatten()
decoypsm_ids = np.asarray(decoypsm_ids).flatten()

targetpsm_peps = np.asarray(targetpeptides)[targetpsm_ids]
decoypsm_peps = np.asarray(decoypeptides)[decoypsm_ids]

#targetpsm_ids = targetpsm_ids[np.argsort(np.asarray(targetpsm_scores).flatten())]
#decoypsm_ids = decoypsm_ids[np.argsort(np.asarray(decoypsm_scores).flatten())]

sorted_targets = np.sort(np.asarray(targetpsm_scores).flatten())[::-1]
sorted_decoys = np.sort(np.asarray(decoypsm_scores).flatten())[::-1]

print(targetpsm_ids[0:10])
print(decoypsm_ids[0:10])

print(sorted_targets[0:100])
print(sorted_decoys[0:100])

#targetpsm_scores = np.column_stack((targetpsm_scores, np.ones(len(targetpsm_scores))))
#decoypsm_scores = np.column_stack((decoypsm_scores, np.zeros(len(decoypsm_scores))))
#decoypsm_scores = np.asarray(decoypsm_scores)

In [ ]:
targetpsm_scores

In [ ]:
df = pd.DataFrame(
    data={'spectrum_ids':np.arange(len(targetpsm_peps)), 'target_psms':targetpsm_peps, 
          'targetpsm_scores':targetpsm_scores, 'decoy_psms':decoypsm_peps, 'decoypsm_scores':decoypsm_scores})
df.to_csv('deepsearch.csv')

In [ ]:
sortedpeps = np.array(sortedpeps)
correct_matches = 0
for ind, psms in enumerate(psm_scores):
    #print('ind: ' + str(ind) + ', mass: ' + str(sortedspecmasses[ind]))
    #print('psms: ' + str(psms) + ', psm mass: ' + str(sortedpepmasses[psms[0]]))
    cand_peps = sortedpeps[psms]
    #print(cand_peps)
    for pep in cand_peps:
        #print('pepidmass: ' + str(pepidmass[pep][0]))
        #print('ids: ' + str(ids[ind]) + ', in: ' + str(pepidmass[pep][0]))
        if ids[ind] in pepidmass[pep][0]:
            correct_matches += 1
            break
print(len(psm_scores))
print(correct_matches)
print(str((correct_matches/len(psm_scores)) * 100) + '% correct matches in top ' + str(num_psm) + ' psms.')

# Scratch Work:

In [ ]:
print(5184 - 1739)

In [ ]:
file = open('/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.target.txt')
tlines = file.readlines()
print(len(tlines))
tpeps = list((tline.split('\t')[0], float(tline.split('\t')[1])) for tline in tlines)
tpeps = sorted(tpeps, key=lambda pair: pair[1])
print(tpeps[10000:10010])
file.close()

In [ ]:
file = open('/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.decoy.txt')
tlines = file.readlines()
print(len(tlines))
tpeps = list((tline.split('\t')[0], float(tline.split('\t')[1])) for tline in tlines)
tpeps = sorted(tpeps, key=lambda pair: pair[1])
print(tpeps[10000:10010])
file.close()

In [ ]:
np.arange(150)

In [20]:
x = torch.ones(2, 3)
z = (torch.ones(3) * 5).view(1, 3)
print(x)
print(z)
# or this
#z = torch.ones(2, 3, 1) * 5
y = torch.cat((x, z), dim=0)
print(y)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[5., 5., 5.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [5., 5., 5.]])


In [34]:
## Keep this cell....
## How to sort a 2D tensor by its first row.....
x = torch.Tensor([5., 3., 4., 1., 2.]).view(1, -1)
z = torch.Tensor([1., 3., 2., 5., 4.]).view(1, -1)
print(x)
print(z)
# or this
#z = torch.ones(2, 3, 1) * 5
y = torch.cat((x, z), dim=0)
_, sorted_index_y = y[0, :].sort()
sorted_y = y[:, sorted_index_y]
print(y)
print(sorted_y)

tensor([[5., 3., 4., 1., 2.]])
tensor([[1., 3., 2., 5., 4.]])
tensor([[5., 3., 4., 1., 2.],
        [1., 3., 2., 5., 4.]])
tensor([[1., 2., 3., 4., 5.],
        [5., 4., 3., 2., 1.]])
